# Symbolische Daten

#### Marcel Lüthi, Departement Mathematik und Informatik, Universität Basel

### Symbole in Scheme

Symbole in Scheme sind ähnlich wie Strings beliebige Charactersequenzen. Im Gegensatz zu String müssen Symbole aber eindeutig sein. 

Beispiele von Symbolen:

In [43]:
'a

'a

In [53]:
'abracadabra

'abracadabra

Das Quote Zeichen ```'``` bezieht sich dabei immer nur auf das nächste Symbol, wie wir hier sehen:

In [60]:
(define a 5)
(define b 7)
(list 'a b)

'(a 7)

Um festzustellen, ob ein bestimmtes Objekt einem Symbol entspricht, nutzen wir das Prädikat ```symbol?```. 

In [62]:
(symbol? 'a)

#t

In [64]:
(symbol? a)

#f

In [70]:
(symbol? "a")

#f

Zusammen mit dem Prädikat ```number?``` können wir nun auch programmatisch zwischen einem Symbol und einem numerischen Wert, der dieses Symol repräsentiert unterscheiden. 

In [72]:
(number? a)

#t

In [74]:
(number? 'a)

#f

In unserem nächsten Anwendungsbeispiel, bei dem wir unseren ersten kleinen Interpreter implementieren, werden wir genau diese Eigenschaft ausnutzen. 

### Anwendungsbeispiel: Symbolische Differentiation

In diesem Anwendungsbeispiel entwickeln wir einen Teil eines Systems zur automatischen Differentiation. Dieses Beispiel illusriert, wie wir mit symbolischen Ausdrücken arbeiten und diese manipulieren können. Hier werden bereits viele Komponenten erklärt, die wir auch später bei der Implementation unseres eigenen Lisp Interpreters nutzen werden. 

Unsere Funktion ```deriv``` nimmt einen Ausdruck abzuleitenden Ausdruck ```exp``` sowie die Variable nach der wir differentieren wollen als Argumente entgegen, und gibt den korrekt abgeleiteten Ausdruck zurück. Wir implementieren hier nur die Summenregel. Die Implementation der Produktregel überlassen wir Ihnen als Übungsaufgabe. 

In [41]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp)
         (if (same-variable? exp var) 1 0))
        ((sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var)))        
        ((product? exp)         
         (make-sum
          (make-product (multiplier exp)
                        (deriv (multiplicand exp) var))
          (make-product (deriv (multiplier exp) var)
                        (multiplicand exp))))
        (else
         (error "unknown expression type -- DERIV" exp))))

Das Grundgerüst ist einfach zu verstehen. 

* Die erste Regel ist die Regel zur Ableitung von Konstanten. Eine Konstante nach einer beliebigen Variable abgeleitet gibt immer 0. 
* Die zweite Regel kommt zur Anwendung, wenn als Ausruck nur eine Variable haben. Dann müssen wir unterscheiden, ob es sich um die Variable handelt nach der wir ableiten wollen, oder eine andere Variable. Wir nutzen dafür die folgenden zwei Hilfsfunktionen:
* Der letzten zwei Fälle treffen ein, wenn wir eine Summe oder ein Produkt differentieren wollen. In diesem Fall werden einfach die entsprechenden Ableitungsformeln angewendet.  

Die obige Implementation nutzt einige Hilfsfunktionen, die zwar intuitiv einfach zu verstehen sind, die wir aber noch definieren müssen. 

Als erstes brauchen wir eine Funktion um zu entscheiden, ob es sich bei einem Ausdruck um eine Variable handelt. Eine Variable definieren wir einfach als Symbol:

In [2]:
(define (variable? v) (symbol? v))

Die nächste Funktion stellt fest, ob die zwei Ausdrücke dieselbe Variable repräsentieren. 

In [3]:
(define (same-variable? v1 v2) (and (variable? v1) (variable? v2) (eq? v1 v2)))

Schlussendlich müssen wir noch definieren, wie wir mit Summen arbeiten.

 Wir repräsentieren eine Summe als Liste, wobei das erste Argument das Symbol ```'+``` ist. Damit können wir erkennen, dass es sich beim Ausdruck um eine Summe handelt. Dies wird in folgendem Konstruktor definiert:

In [4]:
(define (make-sum a1 a2) (list '+ a1 a2)) 

Um zu entscheiden, ob es sich bei einem Ausdruck (einer Liste) um eine Summe handelt, muss das erste Element das Symbol ```'+``` sein gefolgt von einem Paar von weiteren Ausdrücken.  

In [6]:
(define (sum? x)   ; Ist x eine Summe
  (and (pair? x) (eq? (car x) '+)))

Nun definieren wir noch die Zugriffsfunktionen, um auf die beiden Summanden zuzugreifen:

In [8]:
(define (addend s) (cadr s)) ; summand1
(define (augend s) (caddr s)) ; summand2


Und schlussendlich noch dasselbe für Produkte

In [30]:
(define (make-product m1 m2) (list '* m1 m2))

In [31]:
(define (multiplier p) (cadr p))

In [32]:
(define (multiplicand p) (caddr p))

Nun können wir unseren ersten Interpreter ausprobieren:

In [33]:
(deriv '(+ x 3) 'x)

'(+ 1 0)

In [34]:
(deriv '(* x y) 'x)

'(+ (* x 0) (* 1 y))

In [35]:
(deriv '(* (* x y) (+ x 3)) 'x)

'(+ (* (* x y) (+ 1 0)) (* (+ (* x 0) (* 1 y)) (+ x 3)))

#### Miniübung

* Beim obigen Ausdruck entstehen unschöne Ableitungen wie ```'(+ 0 (+ 1 0))```. Wie würden Sie das Programm abändern, so dass dieser zu ```1``` vereinfacht wird?
* Erweitern Sie das Programm um weitere Regeln, wie der Kettenregel


### Typen (Tagged data)

Um zu erkennen, um welchen Type von Ausdruck es sich bei einer Liste handelt, haben wir uns im ersten Element der Liste den Typ als Symbol gespeichert. 

In [42]:
(define (make-sum a1 a2) (cons '+ a1 a2))

Wir können diesen Ansatz generalisieren und mit den folgenden Hilfsfunktionien für die Typisierung von Listen definieren. 

In [44]:
; Typinformation hinzufügen
(define (attach-tag type-tag contents) 
  (cons type-tag contents))

; Auf Typinformation zugreifen
(define (type-tag typed-content)
  (car typed-content))

 ; Auf Inhalt zugreifen
(define (content typed-content) (cdr typed-content))

Nun können wir unsere Daten einfach typisieren:

In [45]:
(define sumExpr (attach-tag '+ (list 'a 'b) ))
(define numberExpr (attach-tag 'number 5 ))
(define variableExpr (attach-tag 'variable 'a))

Unser Programm um Ableitungen zu berechnen sieht dann wie folgt aus:

In [46]:
(define (deriv exp var)
  (cond ((eq? (type-tag exp) 'number) 0)
        ((eq? (type-tag exp) 'variable)
         (if (same-variable? exp var) 1 0))
        ((eq? (type-tag exp) '+)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var)))        
        (else
         (error "unknown expression type -- DERIV" exp))))
(define (same-variable? exp var)
  (eq? (content exp) (content var)))